In [ ]:
import pandas as pd
import numpy as np
import pickle

In [ ]:
df = pd.read_csv('data/source/shakespeare.csv')

In [ ]:
remove = df[(df['Player'].notna()) & (df["PlayerLine"]!='Exeunt')]

In [ ]:
data = remove[['ActSceneLine', "Player", "PlayerLine"]]
data

In [ ]:
class Data_Creator:
    def __init__(self, data):
        #data = pd.read_csv(filepath)
        data = self.clean(data)
        self.tokenized_df = self.tokenize(data)
        

    def tokenize(self, df):
        data = df['text'].apply(lambda x: x.strip().lower().split())
        tokenized = df.assign(PlayerLine = data)
        return tokenized
    
    def clean(self, data):
        
        repl = {'@\w*': ' ', '&amp;' : 'and','\su\s':' you ', '&#\w*;': ' ', 
        '#':' ', '\s2\s': 'two', "ð[^ ]*": ' ' ,
        "â[^ ]*": ' ',"(dont)|(don't)": 'do not', "(cant)|(can't)": "can not",
        "(yous)|(you's)": "you is", "(yous)|(you's)": "you is", 
        "(youve)|(you've)": "you have", "(doesnt)|(doesn't)": 'does not', 
        "(wont)|(won't)": 'will not', "[0-9]+\.*[0-9%]+\w*" : "NUMBER",'\\n\.':' ' ,'\\n':' ',
        "\.{2,}": '.', "!{2,}":'!', "\?{2,}":'?', 'ing[^a-z]':' ', 'ed[^a-z]': ' ', '_':" ",
        ' +': ' ', ',':'', '\\.':' <EOS>'}

        cleaned_tweet = data['PlayerLine'].str.lower()
        cleaned_tweet = cleaned_tweet.replace(repl, regex=True)
        cleaned = data.assign(text = cleaned_tweet)
        return cleaned
    
class vocab_builder:
    def __init__(self, tokenized_df):
        self.longest= 0
        self.idx_word= {}
        self.word_idx = {}
        self.tracker = {}
        tokenized_df['PlayerLine'].apply(vocab_builder.vocab_gen, args=(self,))
        self.word_idx['<PAD>'] = len(self.word_idx)
        self.idx_word[len(self.idx_word)] = '<PAD>'
        self.word_idx['<UNK>'] = len(self.word_idx)
        self.idx_word[len(self.idx_word)] = '<UNK>'
        self.word_idx['<EOS>'] = len(self.word_idx)
        self.idx_word[len(self.idx_word)] = '<EOS>'
        
    def vocab_gen(sentence, self):
        count = 0 
        for word in sentence:
            count +=1
            if word not in self.word_idx:
                self.tracker[word] = 1
                self.word_idx[word] = len(self.word_idx)
                self.idx_word[len(self.idx_word)] = word
            else:
                self.tracker[word] += 1
        if self.longest < count:
            self.longest = count
        return 

In [ ]:
created = Data_Creator(data)
vocab = vocab_builder(created.tokenized_df)

In [ ]:
created.tokenized_df

In [ ]:
info = created.tokenized_df['text']
info.iloc[0]

In [ ]:
len(info)

In [ ]:
train_data = []
info = created.tokenized_df['text']
for i in range(len(info)-1):
    dictionary = {}
    dictionary["src"] = info.iloc[i]
    dictionary["trg"] = info.iloc[i+1]
    train_data.append(dictionary)

In [ ]:
import json

with open('output.jsonl', 'w') as outfile:
    for entry in train_data:
        json.dump(entry, outfile)
        outfile.write('\n')

In [ ]:
vocab.idx_word

In [ ]:
vocab.idx_word.values()

In [ ]:
with open('vocab_list.pickle', 'wb') as handle:
    pickle.dump(vocab.idx_word, handle)